## 关键帧提取

In [33]:
import os
import sys
import glob
import shutil
import codecs
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import numpy as np
import time
from multiprocessing import Pool

%pylab inline
from PIL import Image

Populating the interactive namespace from numpy and matplotlib


In [34]:
IN_PATH = './ucf101_frames'
OUT_PATH = './settings/ucf101'

if not os.path.exists(OUT_PATH):
    os.makedirs('my_folder')


In [35]:
video_file_paths = glob.glob(IN_PATH + '/*')
print(video_file_paths)

['./ucf101_frames/v_SoccerPenalty_g15_c05', './ucf101_frames/v_SoccerJuggling_g13_c02', './ucf101_frames/v_HorseRace_g10_c02', './ucf101_frames/v_FloorGymnastics_g08_c02', './ucf101_frames/v_BoxingPunchingBag_g04_c06', './ucf101_frames/v_Surfing_g02_c01', './ucf101_frames/v_IceDancing_g19_c02', './ucf101_frames/v_LongJump_g06_c03', './ucf101_frames/v_Hammering_g10_c03', './ucf101_frames/v_BodyWeightSquats_g14_c01', './ucf101_frames/v_TableTennisShot_g02_c02', './ucf101_frames/v_SoccerPenalty_g12_c03', './ucf101_frames/v_WallPushups_g14_c03', './ucf101_frames/v_WritingOnBoard_g15_c02', './ucf101_frames/v_HorseRace_g07_c06', './ucf101_frames/v_TennisSwing_g10_c06', './ucf101_frames/v_HorseRace_g20_c03', './ucf101_frames/v_StillRings_g05_c04', './ucf101_frames/v_SumoWrestling_g04_c03', './ucf101_frames/v_Kayaking_g18_c07', './ucf101_frames/v_FieldHockeyPenalty_g25_c04', './ucf101_frames/v_BodyWeightSquats_g04_c02', './ucf101_frames/v_YoYo_g17_c06', './ucf101_frames/v_Rafting_g15_c01', './

In [37]:
video_file_paths_lower = [] 
for v in video_file_paths:
    video_file_paths_lower.append(v.lower())
print(video_file_paths_lower)    

['./ucf101_frames/v_soccerpenalty_g15_c05', './ucf101_frames/v_soccerjuggling_g13_c02', './ucf101_frames/v_horserace_g10_c02', './ucf101_frames/v_floorgymnastics_g08_c02', './ucf101_frames/v_boxingpunchingbag_g04_c06', './ucf101_frames/v_surfing_g02_c01', './ucf101_frames/v_icedancing_g19_c02', './ucf101_frames/v_longjump_g06_c03', './ucf101_frames/v_hammering_g10_c03', './ucf101_frames/v_bodyweightsquats_g14_c01', './ucf101_frames/v_tabletennisshot_g02_c02', './ucf101_frames/v_soccerpenalty_g12_c03', './ucf101_frames/v_wallpushups_g14_c03', './ucf101_frames/v_writingonboard_g15_c02', './ucf101_frames/v_horserace_g07_c06', './ucf101_frames/v_tennisswing_g10_c06', './ucf101_frames/v_horserace_g20_c03', './ucf101_frames/v_stillrings_g05_c04', './ucf101_frames/v_sumowrestling_g04_c03', './ucf101_frames/v_kayaking_g18_c07', './ucf101_frames/v_fieldhockeypenalty_g25_c04', './ucf101_frames/v_bodyweightsquats_g04_c02', './ucf101_frames/v_yoyo_g17_c06', './ucf101_frames/v_rafting_g15_c01', './

In [39]:
classid_fn = 'ucfTrainTestlist/classInd.txt'
f = open(classid_fn, "r")
lines = f.readlines()
f.close() 
print(lines)
classid = {} 
for l in lines: 
    l = l.strip('\n')
    l = l.split()
    id = int(l[0])
    act = l[1].lower()
    classid[act] = id 
print(classid)    

['1 ApplyEyeMakeup\n', '2 ApplyLipstick\n', '3 Archery\n', '4 BabyCrawling\n', '5 BalanceBeam\n', '6 BandMarching\n', '7 BaseballPitch\n', '8 Basketball\n', '9 BasketballDunk\n', '10 BenchPress\n', '11 Biking\n', '12 Billiards\n', '13 BlowDryHair\n', '14 BlowingCandles\n', '15 BodyWeightSquats\n', '16 Bowling\n', '17 BoxingPunchingBag\n', '18 BoxingSpeedBag\n', '19 BreastStroke\n', '20 BrushingTeeth\n', '21 CleanAndJerk\n', '22 CliffDiving\n', '23 CricketBowling\n', '24 CricketShot\n', '25 CuttingInKitchen\n', '26 Diving\n', '27 Drumming\n', '28 Fencing\n', '29 FieldHockeyPenalty\n', '30 FloorGymnastics\n', '31 FrisbeeCatch\n', '32 FrontCrawl\n', '33 GolfSwing\n', '34 Haircut\n', '35 Hammering\n', '36 HammerThrow\n', '37 HandstandPushups\n', '38 HandstandWalking\n', '39 HeadMassage\n', '40 HighJump\n', '41 HorseRace\n', '42 HorseRiding\n', '43 HulaHoop\n', '44 IceDancing\n', '45 JavelinThrow\n', '46 JugglingBalls\n', '47 JumpingJack\n', '48 JumpRope\n', '49 Kayaking\n', '50 Knitting\n'

In [54]:

def create_train_file(infile,outfile): 
    
    f = open(infile, "r")
    lines = f.readlines()
    f.close() 
    
    lines_setting = []
    for l in lines: 
        l = l.strip('\n')
        l = l.split()
        id = int(l[1])-1
        act = l[0]
        
        act = act.split('/')[-1]
        act = act.split('.')[0]
        
        for i in range(len(video_file_paths_lower)):
            if act.lower() in video_file_paths_lower[i]:
                #print(act)
                #print(video_file_paths_lower[i])
                #print(video_file_paths[i])
                act = video_file_paths[i].split('/')[-1]
                #print(act) 
                break 
                
        act_dir = IN_PATH + '/' + act         
        numjpgs = len(glob.glob(act_dir + '/*.jpg')) 
        
        l_s = act + ' ' + str(numjpgs) + ' ' + str(id)
        
        lines_setting.append(l_s)
        #input("debugging")
    
    f = open(outfile, 'w') 
    for l in lines_setting:
        f.write('{}\n'.format(l))
    f.close()    


In [55]:
for i in range(3):
    infile = 'ucfTrainTestlist/trainlist{:02}.txt'.format(i+1)
    outfile = OUT_PATH + '/' + 'train_rgb_split{}.txt'.format(i+1)
    print(infile)
    print(outfile)
    create_train_file(infile,outfile)


ucfTrainTestlist/trainlist01.txt
./settings/ucf101/train_rgb_split1.txt
ucfTrainTestlist/trainlist02.txt
./settings/ucf101/train_rgb_split2.txt
ucfTrainTestlist/trainlist03.txt
./settings/ucf101/train_rgb_split3.txt


In [56]:
def create_test_file(infile,outfile,classid): 
    
    f = open(infile, "r")
    lines = f.readlines()
    f.close() 
    
    lines_setting = []
    for l in lines: 
        l = l.strip('\n')
        l = l.split()
        act = l[0]
        
        act = act.split('/')[-1]
        act = act.split('.')[0]
        
        cla = act.split('_')[1].lower() 

        
        for i in range(len(video_file_paths_lower)):
            if act.lower() in video_file_paths_lower[i]:
                #print(act)
                #print(video_file_paths_lower[i])
                #print(video_file_paths[i])
                act = video_file_paths[i].split('/')[-1]
                #print(act) 
                break 
        
        if cla in classid:
            id = classid[cla]-1
        else:
            print("action not belonging to 101 classes : {}".format(cla))
            id = -1         
        act_dir = IN_PATH + '/' + act 
        
        numjpgs = len(glob.glob(act_dir + '/*.jpg')) 
        l_s = act + ' ' + str(numjpgs) + ' ' + str(id)
        
        lines_setting.append(l_s)
    
    f = open(outfile, 'w') 
    for l in lines_setting:
        f.write('{}\n'.format(l))
    f.close()    


In [57]:
for i in range(3):
    infile = 'ucfTrainTestlist/testlist{:02}.txt'.format(i+1)
    outfile = OUT_PATH + '/' + 'val_rgb_split{}.txt'.format(i+1)
    print(infile)
    print(outfile)
    create_test_file(infile,outfile,classid)

ucfTrainTestlist/testlist01.txt
./settings/ucf101/val_rgb_split1.txt
ucfTrainTestlist/testlist02.txt
./settings/ucf101/val_rgb_split2.txt
ucfTrainTestlist/testlist03.txt
./settings/ucf101/val_rgb_split3.txt


In [58]:
print(classid)

{'applyeyemakeup': 1, 'applylipstick': 2, 'archery': 3, 'babycrawling': 4, 'balancebeam': 5, 'bandmarching': 6, 'baseballpitch': 7, 'basketball': 8, 'basketballdunk': 9, 'benchpress': 10, 'biking': 11, 'billiards': 12, 'blowdryhair': 13, 'blowingcandles': 14, 'bodyweightsquats': 15, 'bowling': 16, 'boxingpunchingbag': 17, 'boxingspeedbag': 18, 'breaststroke': 19, 'brushingteeth': 20, 'cleanandjerk': 21, 'cliffdiving': 22, 'cricketbowling': 23, 'cricketshot': 24, 'cuttinginkitchen': 25, 'diving': 26, 'drumming': 27, 'fencing': 28, 'fieldhockeypenalty': 29, 'floorgymnastics': 30, 'frisbeecatch': 31, 'frontcrawl': 32, 'golfswing': 33, 'haircut': 34, 'hammering': 35, 'hammerthrow': 36, 'handstandpushups': 37, 'handstandwalking': 38, 'headmassage': 39, 'highjump': 40, 'horserace': 41, 'horseriding': 42, 'hulahoop': 43, 'icedancing': 44, 'javelinthrow': 45, 'jugglingballs': 46, 'jumpingjack': 47, 'jumprope': 48, 'kayaking': 49, 'knitting': 50, 'longjump': 51, 'lunges': 52, 'militaryparade': 